In [39]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import time
import numpy as np
import tensorflow as tf
from sklearn import metrics
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [40]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

trainset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

testset = datasets.MNIST('./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset,  batch_size=4, shuffle=True)

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,4,5)
        self.conv2 = nn.Conv2d(4,10,5)
        self.fc1 = nn.Linear(10*4*4, 60)
        self.fc2 = nn.Linear(60,10)
        
        self.dropout = nn.Dropout(0.2)
        
    def forward(self,x):
        x = self.dropout(F.relu(self.conv1(x))) #4,24,24
        x = F.max_pool2d(x, 2, 2) #4,12,12
        x = self.dropout(F.relu(self.conv2(x))) #10,8,8
        x = F.max_pool2d(x, 2, 2) #10,4,4
        x = x.view(-1,10*4*4)
        x = self.dropout(F.relu(self.fc1(x)))
        return F.log_softmax(self.fc2(x), dim=1)

In [46]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = F.nll_loss(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 1000 == 0:
            print('Train Epoch: {:2d} [{:5d}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [47]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target).item() 
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [44]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): print(torch.cuda.get_device_name(0))

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch:  0 [    0/60000 (0%)]	Loss: 2.110638
Train Epoch:  0 [ 4000/60000 (7%)]	Loss: 0.061536
Train Epoch:  0 [ 8000/60000 (13%)]	Loss: 0.034020
Train Epoch:  0 [12000/60000 (20%)]	Loss: 0.391845
Train Epoch:  0 [16000/60000 (27%)]	Loss: 0.002954
Train Epoch:  0 [20000/60000 (33%)]	Loss: 0.029037
Train Epoch:  0 [24000/60000 (40%)]	Loss: 0.000443
Train Epoch:  0 [28000/60000 (47%)]	Loss: 0.469328
Train Epoch:  0 [32000/60000 (53%)]	Loss: 0.010578
Train Epoch:  0 [36000/60000 (60%)]	Loss: 0.486472
Train Epoch:  0 [40000/60000 (67%)]	Loss: 0.004031
Train Epoch:  0 [44000/60000 (73%)]	Loss: 0.727852
Train Epoch:  0 [48000/60000 (80%)]	Loss: 0.000084
Train Epoch:  0 [52000/60000 (87%)]	Loss: 0.011323
Train Epoch:  0 [56000/60000 (93%)]	Loss: 0.190573

Test set: Average loss: 0.0782, Accuracy: 9749/10000 (97%)

Train Epoch:  1 [    0/60000 (0%)]	Loss: 0.000502
Train Epoch:  1 [ 4000/60000 (7%)]	Loss: 0.006451
Train Epoch:  1 [ 8000/60000 (13%)]	Loss: 0.002034
Train Epoch:  1 [12000/60

### Save and Load Model

In [52]:
torch.save(model.state_dict(),"mnist_cnn1.pt")

In [56]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load('mnist_cnn1.pt'))
test(loaded_model, device, test_loader)